In [9]:
from bs4 import BeautifulSoup
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm

#Change this if format requires
file_to_read = pd.read_csv('movies2017.csv')
file_to_read = file_to_read[file_to_read['revenue']!=0]
movies = file_to_read['title'].values.tolist()
len(movies)

853

In [6]:
base_url = "https://www.rottentomatoes.com/search?search="

#Function to create search URLs
def create_search_url(movie_name):
    return base_url + '+'.join(movie_name.split())

# Creating search URLs for each movie
search_urls = {movie: create_search_url(movie) for movie in movies}
print(len(movies))

853


In [8]:
#Get score from website
def get_specific_rotten_tomato_score(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        media_rows = soup.find_all('search-page-media-row', {'data-qa': 'data-row'})

        for row in media_rows:
            if 'tomatometerscore' in row.attrs:
                score = row.attrs['tomatometerscore']
                if score:
                    return int(score)
    return None    


#Iterate through list
movie_scores_specific = {}
for movie, url in search_urls.items():
    score = get_specific_rotten_tomato_score(url)
    movie_scores_specific[movie] = score

#Make df using scores
movies_score_df_specific = pd.DataFrame(list(movie_scores_specific.items()), columns=['Movie', 'Specific Rotten Tomato Score'])
movies_score_df_specific.head(10) 

,Movie,Specific Rotten Tomato Score
0,Logan,None
1,Split,None
2,John Wick: Chapter 2,None
3,Fifty Shades Darker,None
4,xXx: Return of Xander Cage,None
5,Below Her Mouth,None
6,A Dog's Purpose,None
7,The Invisible Guest,None
8,Get Out,None
9,The Lego Batman Movie,None


In [10]:
movies_score_df_specific

,Movie,Specific Rotten Tomato Score
0,Logan,None
1,Split,None
2,John Wick: Chapter 2,None
3,Fifty Shades Darker,None
4,xXx: Return of Xander Cage,None
...,...,...
845,Nan ji zhi lian,None
846,Khaltoor,None
847,Journey to Tataouine,None
848,Gonjeshkake ashi mashi,None


In [11]:
movies_score_df_specific = movies_score_df_specific.dropna(subset=[movies_score_df_specific.columns[1]])

#Output file for current file_to_read
movies_score_df_specific.to_csv('2017scores.csv', index=False)

In [12]:
def combine_dataframes_from_files(file_list):
    dfs = []

    for file_path in file_list:
        df = pd.read_csv(file_path)
        dfs.append(df)

    combined_df = pd.concat(dfs, ignore_index=True)

    return combined_df

#List of output files to combine, CHANGE IF NEW FILES USED
file_list = ['2013scores.csv', '2014scores.csv', '2015scores.csv', '2015scores.csv'
             , '2016scores.csv', '2017scores.csv', '2018scores.csv', '2019scores.csv', '2020scores.csv'
             , '2021scores.csv', '2022scores.csv', '2023scores.csv', 'romescores.csv']

combined_dataframe = combine_dataframes_from_files(file_list)
combined_dataframe = combined_dataframe.drop_duplicates(subset=['Movie'])

combined_dataframe.to_csv('all_scores.csv', index=False)

In [78]:
#Drop all duplicates
duplicates = combined_dataframe['Movie'].duplicated()

num_duplicates = duplicates.sum()
num_duplicates

0